# ArcGIS Python API

- `ArcGIS Python API` empowers users to leverage `Python` programming to interact with `ArcGIS Online` or `ArcGIS Enterprise`, facilitating tasks such as `managing content`, `creating web layers`, `querying data`, and `conducting spatial analysis`.
- [ArcGIS Python API Documentation](https://developers.arcgis.com/python/)
- [ArcGIS Python API Module](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html)


In [1]:
# import the GIS class in gis module
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from time import strftime
import os, requests, zipfile
import pandas as pd
import geopandas as gpd
import arcpy,datetime,json

## Installation and Setup

- Install the `ArcGIS API for Python` using `pip`
- Access `ArcGIS Online` or `ArcGIS Enterprise` by providing your credentials.


## Managing Content

### Connecting to `ArcGIS`


In [2]:
gis = GIS("home")
print("Logged in as " + str(gis.properties.user.username))

Logged in as yxy180050@utdallas.edu_UTDEPPS


### Accessing GIS Content


In [3]:
# List user's content
user_content = gis.content.search(query="*")
user_content[:5]

[<Item title:"Using Python: Homelessness Resource Allocation Project" type:StoryMap owner:tbh160130@utdallas.edu_UTDEPPS>,
 <Item title:"Linear Sampling Toolbox" type:Geoprocessing Sample owner:elvini>,
 <Item title:"Areal Sampling Toolbox" type:Geoprocessing Sample owner:elvini>,
 <Item title:"Kalani's_Personal_Story" type:Web Mapping Application owner:kct200000@utdallas.edu_UTDEPPS>,
 <Item title:"Top 10 Largest Anime Conventions in America" type:StoryMap owner:acj190002@utdallas.edu_UTDEPPS>]

### Uploading Data


In [4]:
    
shapefile_path = "./assets/StorageTankLocations_Active2024_05.zip"
# Search for existing items with the same name
existing_items = gis.content.search(query=f'title:"StorageTankLocations_Active2024_05"')

# Delete existing items
for item in existing_items:
    item.delete()

# Upload the shapefile
try:
    uploaded_item = gis.content.add({}, shapefile_path)
    print("Shapefile uploaded successfully.")
except Exception as e:
    print("Error uploading shapefile:", e)

Shapefile uploaded successfully.


In [5]:
uploaded_item

<Item title:"StorageTankLocations_Active2024_05" type:Shapefile owner:yxy180050@utdallas.edu_UTDEPPS>

## Creating Web Layers

### Publishing Feature Layer


In [6]:
try:
    # Publish a feature layer from a shapefile
    published_item = uploaded_item.publish()
except Exception as e:
    print(e)


In [7]:
published_item = gis.content.search(query= 'StorageTankLocations_Active2024_05',item_type= "Feature Layer")
published_item[0]

<Item title:"StorageTankLocations_Active2024_05" type:Feature Layer Collection owner:yxy180050@utdallas.edu_UTDEPPS>

### Creating Web Map


In [8]:
type(published_item[0])

arcgis.gis.Item

In [9]:
# Create a new web map item with required properties
web_map_properties = {
    'type': 'Web Map',  # Specify the type of item
    'title': 'My Web Map', 
    'snippet': 'Created with ArcGIS Python API'
}
web_map_item = gis.content.add(web_map_properties)

# Get the web map object
web_map = arcgis.mapping.WebMap()
web_map.definition = {'operationalLayers': []}  # Initialize with empty operational layers

web_map.layers

[]

In [10]:
# Add the layer to the web map
web_map.add_layer(published_item[0])

# Save the web map
web_map_item.update(data=web_map)

True

In [11]:
web_map_item

<Item title:"My Web Map" type:Web Map owner:yxy180050@utdallas.edu_UTDEPPS>

## Querying Data

### Querying Feature Layer


In [12]:
type(published_item[0])

query_layer = arcgis.features.FeatureLayer.fromitem(published_item[0])

# Get the fields of the feature layer
fields = query_layer.properties.fields

# Print field names
for field in fields:
    print(field["name"], end= ", ")

FID, FACILITY_I, FACILITY_N, FACILITY_A, FACILITY_1, FACILITY_C, FACILITY_S, FACILITY_Z, COUNTY_ID, FACILITY_2, MUNICIPALI, FACILITY_M, REG_EXPIRA, TANK_OWNER, TANK_OWN_1, TANK_OWN_2, TANK_OWN_3, TANK_OWN_4, TANK_OWN_5, TANK_OWN_6, PRIMARY_FA, SITE_ID, LATITUDE, LONGITUDE, TANK_INFOR, 